In [2]:
import pandas as pd
import numpy as np
import glob

In [3]:
import folium
import time, re
from tqdm import tqdm_notebook as tqdmn

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException, ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

---

# CAPTURA DE DATOS DE GOOGLE MAPS USANDO HERRAMIENTA CHROMEDRIVER

Esta es una de las actividades escenciales para continuar con el proyecto. Se utilizará la herramienta Chrome Driver. Esta herramienta se utliza normalmente para la automatización de despliegue de pruebas para sitios web y aplicaciones. En este caso en especifico lo que se realizo fue automatizar una serie de pasos de un proceso que consistia en:
1. Utilizar el buscador en Google maps algunos de cierta cadena de caracteres.
2. Capturar ciertos parametros de fichero xml que se despliega en al momento de econtrar el sitio indicado.
3. Guardar esta informacion en un fichero csv.

Para el primer paso, utilizado el fichero de registros que tenia Eixos, se construye un parametro que se buscará en el navegador cumpliendo ciertos condiciones. A continuacion se muestra como se realizo este paso:

In [20]:
df = pd.read_csv('../df_sup.csv', usecols=['title','street','number']).head(30)

In [21]:
df.loc[df['title'] == 'Anton Solé', 'title'] = 'Taller Anton Solé'
df['title'] = df['title'].str.replace('F.Espunes','Talleres Espuña')
df.loc[df['title'] == 'Herpa', 'title'] = 'Erpa'
df.loc[df['title'] == 'Emauto S.C', 'number'] = '40'
df.loc[df['title'] == 'Emauto S.C', 'title'] = 'Em Auto S.C.'
df.loc[df['title'] == 'Findal', 'number'] = '128' 
df.loc[df['title'] == 'Findal', 'street'] = 'CL PUIG CERDÁ'
df.loc[df['title'] == 'Tallers Ulibarri', 'title'] = 'Talleres Chicote'

In [22]:
df['street'] = df['street'].str.replace('CL','Carrer')
df['street'] = df['street'].str.replace('PZ','Plaça')
df['street'] = df['street'].str.replace('PS','Passeig')

df['serch_o'] = df['title']+' '+df['street']+', '+df['number']

In [23]:
df.head()

,title,number,street,serch_o
0,Garces Taller,78,Carrer ROCAFORT,"Garces Taller Carrer ROCAFORT, 78"
1,Taller Anton Solé,10,Plaça NAVAS,"Taller Anton Solé Plaça NAVAS, 10"
2,Auto.Carburación,1,Carrer TEODOR BONAPLATA,"Auto.Carburación Carrer TEODOR BONAPLATA, 1"
3,"M.F, Flomart",30,Passeig MONTJUIC,"M.F, Flomart Passeig MONTJUIC, 30"
4,Talleres Espuña,68,Passeig MONTJUIC,"Talleres Espuña Passeig MONTJUIC, 68"


In [26]:
options = webdriver.ChromeOptions()

options.add_argument('headless')

browser = webdriver.Chrome(options=options)

landmarks = df['serch_o']# PS.full_name+' '+PS.address


# We want to remove any '/' character in the names and addresses in the landmarks list (because they'll break URLs) :
landmarks = [i.replace('/', ' ') for i in landmarks]

# These are the empty lists we will populate with the extracted data :
full_name = []
rating = []
total_ratings = []
landmark_cat = []
description = []
address = []
hours = []
lat = []
long = []
    
# Here's the big loop iterating over the landmarks list :
for landmark in tqdmn(landmarks, leave=False, desc='2. Extracting the data') :
    
    # URL making :
    url = 'https://www.google.com/maps/search/' + landmark
    browser.get(url)

    # Waiting for the name of the landmark to load and be visible. If it fails, skip to next one :
    try :
        WebDriverWait(browser,30).until(EC.visibility_of_element_located((By.CLASS_NAME, "section-hero-header-title-title")))
    except (NoSuchElementException, TimeoutException) as e :
        continue
        
    # Extracting the data and putting it into the empty lists we defined earlier :
    try:
        full_name.append(browser.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]').text)
    except NoSuchElementException :
        full_name.append(np.nan)

    try:
        rating.append(browser.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[2]/div/div[1]/span[1]/span/span').text)
    except NoSuchElementException :
        rating.append(np.nan)

    try:
        total_ratings.append(browser.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[2]/div/div[1]/span[2]/span/span[1]/span[2]/span[1]/button').text)
    except NoSuchElementException:
        total_ratings.append(np.nan)

    try:
        landmark_cat.append(browser.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[2]/div/div[2]/span[1]/span[1]/button').text)
    except NoSuchElementException:
        landmark_cat.append(np.nan)
    
    try:
        description.append(browser.find_element_by_css_selector('div[class=section-editorial-quote]').text)
    except NoSuchElementException:
        description.append(np.nan
                          )
    try:
        address.append(browser.find_element_by_css_selector('div[data-tooltip="Copiar la dirección"]').text)
    except NoSuchElementException :
        address.append(np.nan)

    # Here we capture the popular hours for all 7 days starting with Sunday :
    try:
        hours.append([i.get_attribute('aria-label') for i in browser.find_elements_by_xpath("//*[contains(@aria-label, 'hora:')]")])
    except NoSuchElementException:
        hours.append(np.nan)
        
    try:
        coordinates = browser.find_element_by_css_selector('meta[itemprop=image]').get_attribute('content')
        coordinates = coordinates.split('?center=')[1].split('&zoom=')[0].split('%2C')
        lat.append(coordinates[0])
        long.append(coordinates[1])
    except NoSuchElementException:
        lat.append(np.nan)
        long.append(np.nan)

# Closing the Chrome window
browser.close()

In [27]:
HL = pd.DataFrame(data={'full_name':full_name, 'rating':rating, 'total_ratings':total_ratings, 'landmark_category':landmark_cat, 'description':description, 'address':address, 'hours':hours})

In [ ]:
HL.head()

<img src="t_dataRaw.png">

<img src="t_colHour_dataRaw.png">

In [ ]:
(list(HL[HL.full_name == 'MF Flomart'].hours))

In [ ]:
def visit_planner(place_name) :

    # first we make an empty dataframe out of a list of days as index and a list of hours as columns :'Sunday', 
    place_name = pd.DataFrame(index=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'], \
    #place_name = pd.DataFrame(index=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'],  \
                              columns=['06','07','08','09','10','11','12','13','14','15','16','17','18','19','20',
                                       '21','22','23'])    
    
    # and then we return that dataframe
    return place_name

<img src="empty_table.png">

In [ ]:
table = visit_planner("t_table")

# we need to remove the row for Sunday before proceeding
table.drop('Saturday', axis=0, inplace=True)

for hour in list(table):
    table[hour] = re.findall('.{0,29}hora: '+str(hour)+'..', str(list(HL[HL.full_name=='MF Flomart'].hours)))
    table[hour] = [ re.findall('\d+', str(i))[0] for i in tall2[hour] ]
    table[hour] = table[hour].astype(int)

plt.figure(figsize=(15,7))
sns.heatmap(table, cmap='PuBu', linewidths=0.8, annot=True, annot_kws={'fontsize':8, 'alpha':0.8}, fmt='d', square=True,
           cbar=False)

plt.xticks(np.arange(18), list(tall2))
plt.title("Popular times visiting MF Flomart\n", weight='semibold')

# this bit of code ensures the heatmap will not show up truncated with some versions of matplotlib
b, t = plt.ylim()
b += 0.5
t -= 0.5
plt.ylim(b, t);

<img src="filled_table.png">

<img src="show_ftable.png">

---

In [4]:
csv_files = ['hfull_stcl_ldmks_attem19798_succ6095.csv',
             'hfull_stcl_ldmks_attem33872_succ10300.csv',
             'hfull_stcl_ldmks_attem35886_succ1033.csv',
             'hfull_stcl_ldmks_attem36514_succ424.csv',
             'hfull_stcl_ldmks_attem38201_succ559.csv',
             'hfull_stcl_ldmks_attem38849_succ287.csv',
             'hfull_stcl_ldmks_attem39701_succ256.csv',
             'hfull_stcl_ldmks_attem41723_succ662.csv',
             'hfull_stcl_ldmks_attem45541_succ917.csv',
             'hfull_stcl_ldmks_attem47051_succ623.csv',
             'hfull_stcl_ldmks_attem50323_succ1553.csv',
             'hfull_stcl_ldmks_attem50969_succ240.csv',
             'hfull_stcl_ldmks_attem51465_succ151.csv',
             'hfull_stcl_ldmks_attem53551_succ706.csv',
             'hfull_stcl_ldmks_attem59786_succ1368.csv',
             'hfull_stcl_ldmks_attem63950_succ1057.csv',
             'hfull_stcl_ldmks_attem65546_succ463.csv',
             'hfull_stcl_ldmks_attem69434_succ503.csv',
             'hfull_stcl_ldmks_attem69894_succ176.csv']
list_data = []
  
# Escribimos un loop que irá a través de cada uno de los nombres de archivo a través de globbing y el resultado final será la lista dataframes

for filename in csv_files:
    data = pd.read_csv(filename)
    list_data.append(data)

df_concat = pd.concat(list_data,ignore_index=True)
df_concat = df_concat.drop_duplicates().reset_index(drop=True)

In [6]:
comp_df = pd.DataFrame(index = df_concat.index.values, \
                       columns=('count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'))

In [7]:
def visit_planner0_17(place_name) :
    # first we make an empty dataframe out of a list of days as index and a list of hours as columns :
    place_name = pd.DataFrame(index=['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'], \
                              columns=['24','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17'])    
    return place_name

def visit_planner4_21(place_name) :
    # first we make an empty dataframe out of a list of days as index and a list of hours as columns :
    place_name = pd.DataFrame(index=['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'], \
                              columns=['04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21'])    
    return place_name

def visit_planner5_22(place_name) :
    # first we make an empty dataframe out of a list of days as index and a list of hours as columns :
    place_name = pd.DataFrame(index=['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'], \
                              columns=['05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22'])    
    return place_name


def visit_planner6_23(place_name) :
    # first we make an empty dataframe out of a list of days as index and a list of hours as columns :
    place_name = pd.DataFrame(index=['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'], \
                              columns=['06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23'])    
    return place_name

In [8]:
def supZeros(chnk):
    line = str(list(df_concat[df_concat.index == chnk].hours))
    df_concat['hours'][chnk] = re.sub('\:00\)', ')', line)[2:-2]
    
    #lsd = []
    #sd = str(df_concat['hours'][chnk]).split(',')
    #for i in sd:
    #        if bool(re.findall('\:00\)', i)) == True:
    #            lsd.append(sd.index(i))
    #sd = np.delete(sd,lsd)
    #df_concat['hours'][chnk] = sd[2:-2]

In [9]:
def chgTtbyFi(chnk):
    lsd = []
    sd = str(df_concat['hours'][chnk]).split(',')
    for i in sd:
            if bool(re.findall('\: 23\)', i)) == True:
                sd[sd.index(i)] = ' Nivel de ocupación:  0\\xa0% (hora: 05).' 
                print(i, chnk)
    df_concat['hours'][chnk] = sd[2:-2]

In [10]:
def cambioValSpl(chnk):
        sd = str(df_concat['hours'][chnk]).split(',')
        lst = []
        lsd = []
        for i in sd:
            lst.append(len(i))
            if len(i) < 40:
                lsd.append(sd.index(i))
        posMax = lst.index(max(lst))
        sd = str(df_concat['hours'][chnk]).split(',')
        lst=[]
        i = str(re.findall('.{0,10}habitual.{0,11}', str(list(df_concat[df_concat.index==chnk].hours))))[2:-2]
        ii = int(re.findall('\d+',i)[0])
        isw = int(re.findall('\d+',sd[posMax-1])[2])+1
        #'Nivel de ocupación: 29\\xa0% (hora: 16).'
        if isw < 10:
            sd[posMax] = ' \'Nivel de ocupación: '+str(ii)+'\\xa0% (hora: 0'+ str(isw) +').'
        else: 
            sd[posMax] = ' \'Nivel de ocupación: '+str(ii)+'\\xa0% (hora: '+ str(isw) +').'
        sd = np.delete(sd,lsd)
        df_concat['hours'][chnk] = sd

In [11]:
dfConcat_inx = df_concat.index.values
for i in dfConcat_inx:
    if bool(re.search("habitual.",str(list(df_concat[df_concat.index == i].hours)))) == True:
        cambioValSpl(i)

C:\Users\alter\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [12]:
dfConcat_inx = df_concat.index.values
for i in dfConcat_inx:
    
    if bool(re.search("\:00",str(list(df_concat[df_concat.index == i].hours)))) == True:
        supZeros(i)
    #if bool(re.search("habitual.",str(list(df_concat[df_concat.index == i].hours)))) == True:
        #supZeros(i)
    
    #if (len(re.findall("\(hora\: 23\)",str(list(df_concat[df_concat.index == i].hours)))) == 1 and \
    #    len(re.findall("\(hora\: 05\)",str(list(df_concat[df_concat.index == i].hours)))) == 5):
    #    print(i,'enviado...')
    #    chgTtbyFi(i)

C:\Users\alter\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
HL_inx = df_concat.index.values
cntPer= df_concat.hours.apply(lambda x: len(str(x).split(',')))
#cntPer = df_concat.hours.apply(lambda x: len(x.split(',')))
arrInf = np.column_stack((HL_inx,cntPer))
vac = []
iver = []
#sev = False

# visit_plannerD visit_plannerC visit_plannerS


for (i, j) in arrInf:
    if j >= 90:
        
        sev = False
        six = False
        fiv = False
        
        #SIETE DIAS
        if (len(re.findall('.{0,36}04'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and \
            len(re.findall('.{0,36}21'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and 
            len(re.findall('.{0,36}05'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and 
            len(re.findall('.{0,36}06'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and
            len(re.findall('.{0,36}07'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 ):
            #print(i, j,'tabla de seis dias de 05 a 10')
            vac.append(i)
            t_table = visit_planner4_21("temp_table")
            for hour in list(t_table):
                
                t_table[hour] = re.findall('.{0,36}'+hour+'\)', str(list(df_concat[df_concat.index == i].hours)))
                t_table[hour] = [ re.findall('\d+', str(j))[0] for j in t_table[hour] ]
                t_table[hour] = t_table[hour].astype(int)
            table_des = pd.DataFrame()
            table_des = pd.DataFrame((t_table.describe()[:]).mean(axis=1)).T
            comp_df.iloc[i] = table_des.iloc[-1] 
            sev = True
            
        if (len(re.findall('.{0,36}05'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and \
            len(re.findall('.{0,36}22'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and \
            len(re.findall('.{0,36}06'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and \
            len(re.findall('.{0,36}07'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and sev == False):
            #print(i, j,'tabla de seis dias de 05 a 10')
            vac.append(i)
            t_table = visit_planner5_22("temp_table")
            for hour in list(t_table):
                
                t_table[hour] = re.findall('.{0,36}'+hour+'\)', str(list(df_concat[df_concat.index == i].hours)))
                t_table[hour] = [ re.findall('\d+', str(j))[0] for j in t_table[hour] ]
                t_table[hour] = t_table[hour].astype(int)
            table_des = pd.DataFrame()
            table_des = pd.DataFrame((t_table.describe()[:]).mean(axis=1)).T
            comp_df.iloc[i] = table_des.iloc[-1] 
            sev = True
            
        if (len(re.findall('.{0,36}06'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and \
            len(re.findall('.{0,36}23'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and 
            len(re.findall('.{0,36}07'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and 
            len(re.findall('.{0,36}22'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and sev == False):
            #print(i, j,'tabla de seis dias de 06 a 11')
            vac.append(i)
            t_table = visit_planner6_23("temp_table")
            for hour in list(t_table):
                
                t_table[hour] = re.findall('.{0,36}'+hour+'\)', str(list(df_concat[df_concat.index == i].hours)))
                t_table[hour] = [ re.findall('\d+', str(j))[0] for j in t_table[hour] ]
                t_table[hour] = t_table[hour].astype(int)
            table_des = pd.DataFrame()
            table_des = pd.DataFrame((t_table.describe()[:]).mean(axis=1)).T
            comp_df.iloc[i] = table_des.iloc[-1]   
            sev = True
                    
        if (len(re.findall('.{0,36}24'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and \
            len(re.findall('.{0,36}03'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and \
            len(re.findall('.{0,36}17'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and 
            len(re.findall('.{0,36}05'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and 
            len(re.findall('.{0,36}06'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and
            len(re.findall('.{0,36}07'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and sev == False):
            #print(i, j,'tabla de seis dias de 06 a 11')
            vac.append(i)
            t_table = visit_planner0_17("temp_table")
            for hour in list(t_table):
                
                t_table[hour] = re.findall('.{0,36}'+hour+'\)', str(list(df_concat[df_concat.index == i].hours)))
                t_table[hour] = [ re.findall('\d+', str(j))[0] for j in t_table[hour] ]
                t_table[hour] = t_table[hour].astype(int)
            table_des = pd.DataFrame()
            table_des = pd.DataFrame((t_table.describe()[:]).mean(axis=1)).T
            comp_df.iloc[i] = table_des.iloc[-1]           
            sev = True
            
        #SEIS DIAS            
        if (len(re.findall('.{0,36}04'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}21'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}05'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}06'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}07'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and sev == False ):
            #print(i, j,'tabla de seis dias de 05 a 10')
            vac.append(i)
            t_table = visit_planner4_21("temp_table")
            t_table.drop('Sunday', axis=0, inplace=True)
            for hour in list(t_table):
                
                t_table[hour] = re.findall('.{0,36}'+hour+'\)', str(list(df_concat[df_concat.index == i].hours)))
                t_table[hour] = [ re.findall('\d+', str(j))[0] for j in t_table[hour] ]
                t_table[hour] = t_table[hour].astype(int)
            table_des = pd.DataFrame()
            table_des = pd.DataFrame((t_table.describe()[:]).mean(axis=1)).T
            comp_df.iloc[i] = table_des.iloc[-1]
            
            six = True

        if (len(re.findall('.{0,36}05'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}22'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}06'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}07'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and sev == False and six == False ):
            #print(i, j,'tabla de seis dias de 05 a 10')
            vac.append(i)
            
            t_table = visit_planner5_22("temp_table")
            t_table.drop('Sunday', axis=0, inplace=True)
            for hour in list(t_table):
                
                t_table[hour] = re.findall('.{0,36}'+hour+'\)', str(list(df_concat[df_concat.index == i].hours)))
                t_table[hour] = [ re.findall('\d+', str(j))[0] for j in t_table[hour] ]
                t_table[hour] = t_table[hour].astype(int)
            table_des = pd.DataFrame()
            table_des = pd.DataFrame((t_table.describe()[:]).mean(axis=1)).T
            comp_df.iloc[i] = table_des.iloc[-1]
            
            six = True
            
        if (len(re.findall('.{0,36}06'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}23'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}07'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}22'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and sev == False and six == False ):
            #print(i, j,'tabla de seis dias de 06 a 11')
            vac.append(i)
            
            t_table = visit_planner6_23("temp_table")
            t_table.drop('Sunday', axis=0, inplace=True)
            for hour in list(t_table):
                
                t_table[hour] = re.findall('.{0,36}'+hour+'\)', str(list(df_concat[df_concat.index == i].hours)))
                t_table[hour] = [ re.findall('\d+', str(j))[0] for j in t_table[hour] ]
                t_table[hour] = t_table[hour].astype(int)
            table_des = pd.DataFrame()
            table_des = pd.DataFrame((t_table.describe()[:]).mean(axis=1)).T
            comp_df.iloc[i] = table_des.iloc[-1]
            
            six = True
            
        if (len(re.findall('.{0,36}24'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}03'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}17'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}05'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}06'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}07'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and sev == False and six == False ):
            #print(i, j,'tabla de seis dias de 06 a 11')
            vac.append(i)
            t_table = visit_planner0_17("temp_table")
            t_table.drop('Sunday', axis=0, inplace=True)
            for hour in list(t_table):
                
                t_table[hour] = re.findall('.{0,36}'+hour+'\)', str(list(df_concat[df_concat.index == i].hours)))
                t_table[hour] = [ re.findall('\d+', str(j))[0] for j in t_table[hour] ]
                t_table[hour] = t_table[hour].astype(int)
            table_des = pd.DataFrame()
            table_des = pd.DataFrame((t_table.describe()[:]).mean(axis=1)).T
            comp_df.iloc[i] = table_des.iloc[-1]
            
            six = True
            
        #CINCO DIAS
        if (len(re.findall('.{0,36}04'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}21'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}05'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}06'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}07'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and sev == False and six == False):
            #print(i, j,'tabla de seis dias de 05 a 10')
            vac.append(i)
            t_table = visit_planner4_21("temp_table")
            t_table.drop(['Sunday','Saturday'], axis=0, inplace=True)
            for hour in list(t_table):
                
                t_table[hour] = re.findall('.{0,36}'+hour+'\)', str(list(df_concat[df_concat.index == i].hours)))
                t_table[hour] = [ re.findall('\d+', str(j))[0] for j in t_table[hour] ]
                t_table[hour] = t_table[hour].astype(int)
            table_des = pd.DataFrame()
            table_des = pd.DataFrame((t_table.describe()[:]).mean(axis=1)).T
            comp_df.iloc[i] = table_des.iloc[-1]
            fiv = True
            
        if (len(re.findall('.{0,36}05'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}22'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}06'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}07'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and sev == False and six == False and fiv == False):
            #print(i, j,'tabla de cinco dias de 05 a 10')
            vac.append(i)
            t_table = visit_planner5_22("temp_table")
            t_table.drop(['Sunday','Saturday'], axis=0, inplace=True)
            for hour in list(t_table):
                
                t_table[hour] = re.findall('.{0,36}'+hour+'\)', str(list(df_concat[df_concat.index == i].hours)))
                t_table[hour] = [ re.findall('\d+', str(j))[0] for j in t_table[hour] ]
                t_table[hour] = t_table[hour].astype(int)
            table_des = pd.DataFrame()
            table_des = pd.DataFrame((t_table.describe()[:]).mean(axis=1)).T
            comp_df.iloc[i] = table_des.iloc[-1]
            fiv = True
            
        if (len(re.findall('.{0,36}06'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}23'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}07'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}22'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and sev == False and six == False and fiv == False):
            #print(i, j,'tabla de cinco dias de 06 a 11')
            vac.append(i)
            t_table = visit_planner6_23("temp_table")
            t_table.drop(['Sunday','Saturday'], axis=0, inplace=True)
            for hour in list(t_table):
                
                t_table[hour] = re.findall('.{0,36}'+hour+'\)', str(list(df_concat[df_concat.index == i].hours)))
                t_table[hour] = [ re.findall('\d+', str(j))[0] for j in t_table[hour] ]
                t_table[hour] = t_table[hour].astype(int)
            table_des = pd.DataFrame()
            table_des = pd.DataFrame((t_table.describe()[:]).mean(axis=1)).T
            comp_df.iloc[i] = table_des.iloc[-1]
            fiv = True
        if (len(re.findall('.{0,36}24'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}03'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}17'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}05'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}06'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}07'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and sev == False and six == False and fiv == False):
            vac.append(i)
            t_table = visit_planner0_17("temp_table")
            t_table.drop(['Sunday','Saturday'], axis=0, inplace=True)
            for hour in list(t_table):
                
                t_table[hour] = re.findall('.{0,36}'+hour+'\)', str(list(df_concat[df_concat.index == i].hours)))
                t_table[hour] = [ re.findall('\d+', str(j))[0] for j in t_table[hour] ]
                t_table[hour] = t_table[hour].astype(int)
            table_des = pd.DataFrame()
            table_des = pd.DataFrame((t_table.describe()[:]).mean(axis=1)).T
            comp_df.iloc[i] = table_des.iloc[-1]
        if(fiv == True or six == True or sev == True):
            iver.append(i)
len(vac)
        #bool(re.findall('\: 23|\: 00|\: 01|\: 02|\(hora\:\ \)', i)) == True:
        #re.findall('.{0,36}22'+'\)', str(list(df_concat[df_concat.index == 3227].hours)))
        #for hour in list(t_table):
         #    re.findall('.{0,36}'+hour+'\)', str(list(HL[HL.index == position].hours)))
         #   [ re.findall('\d+', str(i))[0] for i in t_table[hour] ]
         #   t_table[hour].astype(int)

5470

In [14]:
comp_df

,count,mean,std,min,25%,50%,75%,max
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,18.9444,8.10061,10.5556,14.1667,17.5,21.9444,30.5556
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
21521,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21523,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
final_GoogleMaps = pd.merge(df_concat, comp_df, left_index=True, right_index=True, how='outer')
final_GoogleMaps

,full_name,rating,total_ratings,landmark_category,description,address,phone,website,hours,latitude,longitude,count,mean,std,min,25%,50%,75%,max
0,Tallers Garcés,"4,1",(10),Taller de automóviles,NaN,"Carrer de Rocafort, 78, 08015 Barcelona",934 23 10 93,NaN,[],41.379371,2.153754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Taller Antón Solé,NaN,NaN,NaN,NaN,"Plaça de las Navas, 10, 08004 Barcelona",934 23 45 63,NaN,[],41.374035,2.158775,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Auto Carburacion e Inyeccion,"5,0",(2),Taller de reparación de vehículos todoterreno,NaN,"C/ d'Entença, 20, 08015 Barcelona",933 25 46 94,NaN,[],41.376014,2.155615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MF Flomart,"4,9",(35),Tienda de repuestos para automóviles,NaN,"Passeig de Montjuïc, 30, 08004 Barcelona",934 41 13 48,NaN,"['Nivel de ocupación: \xa0% (hora: ).', 'Nivel...",41.372421,2.171510,5,18.9444,8.10061,10.5556,14.1667,17.5,21.9444,30.5556
4,Talleres Espuña,"4,6",(10),Taller de reparación de automóviles,NaN,"Passeig de Montjuïc, 68, 08004 Barcelona",934 41 48 10,NaN,[],41.371756,2.167712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21521,De Pata Negra,"3,7",(264),Bar,NaN,"Plaça de las Navas, 7, 08004 Barcelona, España",NaN,m.facebook.com,[],41.373856,2.159229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21522,Malabida,"4,5",(171),Bar restaurante,NaN,"Carrer de Blai, 63, 08004 Barcelona, España",+34 931 75 81 79,malabida.business.site,"['Nivel de ocupación: 0\\xa0% (hora: 04).', 'N...",41.374590,2.161894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21523,Bodega 1900,"4,4",(990),Bar de tapas,NaN,"Carrer de Tamarit, 91, 08015 Barcelona, España",+34 933 25 26 59,elbarri.com,[],41.375552,2.156562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21524,Vinoteca San Antoni By Wine Palace,"4,5",(68),Bodega,NaN,"Carrer del Comte Borrell, 30, 08015 Barcelona,...",+34 935 39 40 02,winepalace.es,"['Nivel de ocupación: \\xa0% (hora: ).', 'Nive...",41.376737,2.163638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
final_GoogleMaps.to_csv('table_of_MEANS_googleData.csv', index=False)